In [58]:
import pandas as pd
import requests
import base64
from datetime import datetime, timedelta


api_key = "9BBBELQU69JG4JZNMHKNC8WWJ" 
location = "Berlin"
start_date = datetime(2013,5,15)
end_date = datetime(2023,5,15)


url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{start_date.strftime("%Y-%m-%d")}/{end_date.strftime("%Y-%m-%d")}?unitGroup=metric&include=days&key={api_key}&contentType=json"
response = requests.get(url)
if response.status_code == 200:
    data = response.json()

df = pd.read_json(data)

df.to_csv("weather_data_From_5_YearsAgo.csv", index=False)



In [59]:
tbl1 = pd.read_csv("weather_data_From_5_YearsAgo.csv.csv")
tbl2 = pd.read_csv("Last 10 years weather.csv")


In [60]:
tbl2 = tbl2.iloc[14:]


In [61]:
len(tbl1)

3653

In [62]:
len(tbl2)

3653

In [63]:
new_df = pd.DataFrame(columns=["location", "date", "Temp_Max", "Temp_Min", "Temp_Ave", "Feels_Like", "humidity", "Snow", "Rain", "Wind_Speed", "Cloud_Conver", "Sunrise", "Sunset", "Description"])


In [64]:
def calculate_average_date(date1, date2):
    date2 = date2 + ':00'
    dt1 = datetime.strptime(date1, '%Y-%m-%dT%H:%M:%S')
    dt2 = datetime.strptime(date2, '%Y-%m-%dT%H:%M:%S')

    time_diff = dt2 - dt1
    average_dt = dt1 + (time_diff / 2)
    return average_dt.strftime('%H:%M:%S')

In [65]:
new_df = pd.DataFrame(columns=["location", "date", "Temp_Max", "Temp_Min", "Temp_Ave", "Feels_Like", "humidity", "Snow", "Rain", "Wind_Speed", "Cloud_Conver", "Sunrise", "Sunset", "Description"])

for index, row in tbl1.iterrows():
    rowsTbl2 = tbl2.iloc[index]
    if datetime.strptime(row['datetime'], '%Y-%m-%d').date() == datetime.strptime(rowsTbl2["time"], '%d/%m/%Y').date():
        location = row['name']
        date = row['datetime']
        temp_max = int(row['tempmax']) + int(rowsTbl2["temperature_2m_max (°C)"]) / 2 
        temp_min = int(row['tempmin']) + int(rowsTbl2["temperature_2m_min (°C)"]) / 2
        temp_ave = int(row['temp']) + int(rowsTbl2["temperature_2m_mean (°C)"]) / 2
        feels_like = int(row['feelslike'])
        humidity = int(row['humidity'])
        snow = int(row['snow']) + int(rowsTbl2['snowfall_sum (cm)']) / 2
        rain = int(row['precip']) - int(row['snow']) + int(rowsTbl2['rain_sum (mm)']) / 2
        wind_speed = int(row['windspeed']) + int(rowsTbl2['windspeed_10m_max (km/h)']) / 2
        cloud_cover = str(row['cloudcover']) + '%'
        sunrise = calculate_average_date(row['sunrise'], rowsTbl2["sunrise (iso8601)"])
        sunset = calculate_average_date(row['sunset'], rowsTbl2["sunset (iso8601)"])
        description = row['description']
    else:
        location = row['name']
        date = row['datetime']
        temp_max = row['tempmax']
        temp_min = row['tempmin']
        temp_ave = row['temp']
        feels_like = row['feelslike']
        humidity = row['humidity']
        snow = row['snow']
        rain = row['precip']
        wind_speed = row['windspeed']
        cloud_cover = str(row['cloudcover']) + '%'
        sunrise = row['sunrise']
        sunset = row['sunset']
        description = row['description']
    new_df.loc[len(new_df.index)] = [location,date,temp_max,temp_min,temp_ave,feels_like,humidity,snow,rain,wind_speed,cloud_cover,sunrise,sunset,description] 


In [66]:
new_df.to_csv("Merge weather.csv")